In [1]:
client_id = "B23NKMRLJ0HWOQQBKDEREME22SG2F4UENWJUS30PB34XCZYW"
client_secret = "IS0F34JPRNNZFIPBA4AD1HB4PMOVESIOLGASTSD0E0DV05R0"

In [2]:
import foursquare
import requests

In [3]:
client = client = foursquare.Foursquare(client_id=client_id, client_secret=client_secret)

In [135]:
## client.venues('40a55d80f964a52020f31ee3')


In [136]:
## client.venues.explore({'ll':'40.74224,-73.99386'})

In [7]:
categories = client.venues.categories()

In [11]:
import pandas as pd
categories = pd.DataFrame(categories)

In [20]:
categories = client.venues.categories()
## categories['categories']

In [13]:
nightlife_id = list(categories[categories['shortName'] == "Nightlife"]['id'])
nightlife_id = nightlife_id[0]

In [14]:
nightlife_id

'4d4b7105d754a06376d81259'

In [15]:
rest_id = list(categories[categories['shortName'] == 'Food']['id'])
rest_id = rest_id[0]
rest_id

'4d4b7105d754a06374d81259'

In [17]:
rest_id in nightlife_id ## They look similar, but they're different

False

In [24]:
## Getting nightlife venues near Toronto
ll = '43.65479,-79.40263693'
radius = '1000'
limit = '50'

bars = client.venues.explore({'ll':ll, 'radius':radius, 'limit':limit, 'categoryId': nightlife_id})




In [28]:
bars.keys()

dict_keys(['suggestedFilters', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [39]:
len(bars['groups'][0]['items']) ## this key contains all the results we need

50

In [46]:
bars['groups'][0]['items'][0].keys()

dict_keys(['reasons', 'venue', 'referralId'])

In [52]:
bars['groups'][0]['items'][0]['venue'] ## This gives us an overview, but no ratings

{'id': '4df2e93ad22d8c5470dc3b01',
 'name': 'Poetry Jazz Cafe',
 'location': {'address': '224 Augusta Ave.',
  'lat': 43.654974536398164,
  'lng': -79.40237067545584,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.654974536398164,
    'lng': -79.40237067545584}],
  'distance': 29,
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['224 Augusta Ave.', 'Toronto ON', 'Canada']},
 'categories': [{'id': '4bf58dd8d48988d1e7931735',
   'name': 'Jazz Club',
   'pluralName': 'Jazz Clubs',
   'shortName': 'Jazz Club',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_jazzclub_',
    'suffix': '.png'},
   'primary': True}],
 'photos': {'count': 0, 'groups': []}}

In [55]:
## retain venueID to use for details, but first compile the table

venueid = []
name = []
lat = []
long = []

for i in range(len(bars['groups'][0]['items'])):
    id_val = bars['groups'][0]['items'][i]['venue']['id']
    name_val = bars['groups'][0]['items'][i]['venue']['name']
    lat_val = bars['groups'][0]['items'][i]['venue']['location']['lat']
    long_val = bars['groups'][0]['items'][i]['venue']['location']['lng']
    
    venueid.append(id_val)
    name.append(name_val)
    lat.append(lat_val)
    long.append(long_val)

In [57]:
bars_dict = {'id':venueid, 'name':name, 'lat':lat, 'long':long}
bars_df = pd.DataFrame(bars_dict)

In [59]:
bars_df.head()

,id,name,lat,long
0,4df2e93ad22d8c5470dc3b01,Poetry Jazz Cafe,43.654975,-79.402371
1,4e9a55b68231878c156f04bf,Cold Tea,43.654193,-79.401075
2,589bc46a739d8546efb5f2fd,Grey Gardens,43.653479,-79.401427
3,573cf7fb498ea0619b6c64dd,El Rey,43.652764,-79.400048
4,4f25a676e4b01b0c60b7f9c4,Thirsty and Miserable,43.654565,-79.401583


In [60]:
## To use later
barids = venueid

In [62]:
## Repeat process for restaurants
ll = '43.65479,-79.40263693'
radius = '1000'
limit = '50'

rests = client.venues.explore({'ll':ll, 'radius':radius, 'limit':limit, 'categoryId': rest_id})

In [63]:
venueid = []
name = []
lat = []
long = []

for i in range(len(rests['groups'][0]['items'])):
    id_val = rests['groups'][0]['items'][i]['venue']['id']
    name_val = rests['groups'][0]['items'][i]['venue']['name']
    lat_val = rests['groups'][0]['items'][i]['venue']['location']['lat']
    long_val = rests['groups'][0]['items'][i]['venue']['location']['lng']
    
    venueid.append(id_val)
    name.append(name_val)
    lat.append(lat_val)
    long.append(long_val)

In [64]:
rests_dict = {'id':venueid, 'name':name, 'lat':lat, 'long':long}
rests_df = pd.DataFrame(rests_dict)

In [65]:
rests_df.head()

,id,name,lat,long
0,50427a03e4b08d9f5931f593,Seven Lives - Tacos y Mariscos,43.654418,-79.400545
1,4a8d5b48f964a520840f20e3,The Moonbean Cafe,43.654147,-79.400182
2,535163cf498ea10a3b9582b5,Blackbird Baking Co,43.654764,-79.400566
3,55820db4498e4abb523622bf,Otto's Berlin Döner,43.656387,-79.402788
4,4aedf32af964a52091d021e3,Golden Patty,43.654659,-79.401179


In [66]:
restids = venueid

In [67]:
## Getting details on the venues

## Test
restid_sample = restids[:3]

In [69]:
restid_sample[0]

'50427a03e4b08d9f5931f593'

In [75]:
likes = client.venues.likes(restid_sample[0])

In [76]:
likes['likes']['count']

302

In [77]:
rest_likes = []

for i in range(len(restids)):
    venue_call = client.venues.likes(restids[i])
    like_val = venue_call['likes']['count']
    
    rest_likes.append(like_val)

In [78]:
rest_likes[:5]

[302, 150, 62, 121, 28]

In [79]:
rests_df['likes'] = rest_likes

In [81]:
rests_df.head()

,id,name,lat,long,likes
0,50427a03e4b08d9f5931f593,Seven Lives - Tacos y Mariscos,43.654418,-79.400545,302
1,4a8d5b48f964a520840f20e3,The Moonbean Cafe,43.654147,-79.400182,150
2,535163cf498ea10a3b9582b5,Blackbird Baking Co,43.654764,-79.400566,62
3,55820db4498e4abb523622bf,Otto's Berlin Döner,43.656387,-79.402788,121
4,4aedf32af964a52091d021e3,Golden Patty,43.654659,-79.401179,28


In [82]:
bar_likes = []

for i in range(len(barids)):
    venue_call = client.venues.likes(barids[i])
    like_val = venue_call['likes']['count']
    
    bar_likes.append(like_val)

In [83]:
bar_likes[:3]

[28, 162, 39]

In [84]:
bars_df['likes'] = bar_likes

In [85]:
bars_df.head()

,id,name,lat,long,likes
0,4df2e93ad22d8c5470dc3b01,Poetry Jazz Cafe,43.654975,-79.402371,28
1,4e9a55b68231878c156f04bf,Cold Tea,43.654193,-79.401075,162
2,589bc46a739d8546efb5f2fd,Grey Gardens,43.653479,-79.401427,39
3,573cf7fb498ea0619b6c64dd,El Rey,43.652764,-79.400048,71
4,4f25a676e4b01b0c60b7f9c4,Thirsty and Miserable,43.654565,-79.401583,67


In [137]:
## Map visualization using Folium
import folium

map_to = folium.Map(location=[43.6537383,-79.4047757], zoom_start=15)
for lat, long, bars in zip(bars_df['lat'], bars_df['long'], bars_df['name']):
    label = '{}'.format(bars)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_to)

In [138]:
for lat, long, rests in zip(rests_df['lat'], rests_df['long'], rests_df['name']):
    label = '{}'.format(rests)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc4b31',
        fill_opacity=0.5,
        parse_html=False).add_to(map_to)  

In [139]:
map_to

In [90]:
POIs_df = bars_df.append(rests_df)

In [95]:
POIs_df = POIs_df.sort_values('likes', ascending=False)

In [96]:
POIs_df

,id,name,lat,long,likes
8,4ad4c05df964a5204cf620e3,Sneaky Dee's Restaurant & Concert Venue,43.656349,-79.407646,407
0,50427a03e4b08d9f5931f593,Seven Lives - Tacos y Mariscos,43.654418,-79.400545,302
46,54e0dc87498ecbdb39a0fade,Bar Raval,43.655904,-79.410125,266
31,4ad4c05cf964a52007f620e3,Horseshoe Tavern,43.649339,-79.395976,223
9,51438b33e4b0a40e33fe5e77,Jimmy's Coffee,43.654493,-79.401311,200
...,...,...,...,...,...
46,4ad4c05df964a52076f620e3,El Rancho Restaurant & Night Club,43.656673,-79.407101,6
35,4b7e02cbf964a520e1de2fe3,Wild Indigo,43.655090,-79.414315,5
42,4af63f31f964a520980222e3,Sotto Voce,43.655240,-79.413833,5
43,58c9c7b2084be5199d1557a7,Reverie At Weldon Park,43.655297,-79.412642,5


In [98]:
import sqlite3
conn = sqlite3.connect('mini_project_foursquare.db')
c = conn.cursor()

In [100]:
c.execute("""CREATE TABLE Foursquare_Overview (
            id PRIMARY KEY,
            name varchar(255),
            lat decimal(18,2),
            long decimal(18,2),
            likes INT)""")

In [102]:
POIs_df.to_sql('Foursquare_Overview', conn, if_exists='replace', index = False)

In [103]:
c.execute(''' SELECT * FROM Foursquare_Overview LIMIT 10''')
for row in c.fetchall():
     print (row)

('4ad4c05df964a5204cf620e3', "Sneaky Dee's Restaurant & Concert Venue", 43.65634906243686, -79.40764590401695, 407)
('50427a03e4b08d9f5931f593', 'Seven Lives - Tacos y Mariscos', 43.65441817160925, -79.40054467846261, 302)
('54e0dc87498ecbdb39a0fade', 'Bar Raval', 43.65590380020168, -79.41012528837247, 266)
('4ad4c05cf964a52007f620e3', 'Horseshoe Tavern', 43.6493390929762, -79.3959763512841, 223)
('51438b33e4b0a40e33fe5e77', "Jimmy's Coffee", 43.65449315540114, -79.40131090393002, 200)
('4d3e1a66457cb60c7ad5fda4', '416 Snack Bar', 43.64751101216232, -79.40401784574094, 187)
('4ade61b9f964a520797521e3', 'Tequila Bookworm', 43.64769690944723, -79.40154896874483, 184)
('52c5dd8d498e25dee9f19457', 'Isshin Ramen 一心', 43.65651655724417, -79.40694540739058, 179)
('4e9a55b68231878c156f04bf', 'Cold Tea', 43.65419313416817, -79.40107491108753, 162)
('4ad7bf71f964a5205b0e21e3', 'Sin and Redemption', 43.65466135819523, -79.39173728227614, 153)


In [107]:
c.execute('DROP TABLE Foursquare_Overview')

In [108]:
POIs_df = POIs_df.drop("id", axis=1)

In [109]:
POIs_df.head(3)

,name,lat,long,likes
8,Sneaky Dee's Restaurant & Concert Venue,43.656349,-79.407646,407
0,Seven Lives - Tacos y Mariscos,43.654418,-79.400545,302
46,Bar Raval,43.655904,-79.410125,266


In [110]:
c.execute("""CREATE TABLE Foursquare_Overview (
            id PRIMARY KEY,
            name varchar(255),
            lat decimal(18,2),
            long decimal(18,2),
            likes INT)""")

In [112]:
POIs_df.to_sql('Foursquare_Overview', conn, if_exists='replace', index = False)

In [113]:
c.execute(''' SELECT * FROM Foursquare_Overview LIMIT 10''')
for row in c.fetchall():
     print (row)

("Sneaky Dee's Restaurant & Concert Venue", 43.65634906243686, -79.40764590401695, 407)
('Seven Lives - Tacos y Mariscos', 43.65441817160925, -79.40054467846261, 302)
('Bar Raval', 43.65590380020168, -79.41012528837247, 266)
('Horseshoe Tavern', 43.6493390929762, -79.3959763512841, 223)
("Jimmy's Coffee", 43.65449315540114, -79.40131090393002, 200)
('416 Snack Bar', 43.64751101216232, -79.40401784574094, 187)
('Tequila Bookworm', 43.64769690944723, -79.40154896874483, 184)
('Isshin Ramen 一心', 43.65651655724417, -79.40694540739058, 179)
('Cold Tea', 43.65419313416817, -79.40107491108753, 162)
('Sin and Redemption', 43.65466135819523, -79.39173728227614, 153)


In [117]:
POIs_df.head(10)

,name,lat,long,likes
8,Sneaky Dee's Restaurant & Concert Venue,43.656349,-79.407646,407
0,Seven Lives - Tacos y Mariscos,43.654418,-79.400545,302
46,Bar Raval,43.655904,-79.410125,266
31,Horseshoe Tavern,43.649339,-79.395976,223
9,Jimmy's Coffee,43.654493,-79.401311,200
29,416 Snack Bar,43.647511,-79.404018,187
19,Tequila Bookworm,43.647697,-79.401549,184
24,Isshin Ramen 一心,43.656517,-79.406945,179
1,Cold Tea,43.654193,-79.401075,162
44,Sin and Redemption,43.654661,-79.391737,153


In [119]:
bars_df['Category'] = 'Nightlife'

In [120]:
rests_df['Category'] = 'Food'

In [121]:
POIs_df = bars_df.append(rests_df)

In [131]:
POIs_df = POIs_df.sort_values('likes', ascending=False)

In [133]:
POIs_df.head(10)

,name,lat,long,likes,Category
8,Sneaky Dee's Restaurant & Concert Venue,43.656349,-79.407646,407,Nightlife
0,Seven Lives - Tacos y Mariscos,43.654418,-79.400545,302,Food
46,Bar Raval,43.655904,-79.410125,266,Food
31,Horseshoe Tavern,43.649339,-79.395976,223,Nightlife
9,Jimmy's Coffee,43.654493,-79.401311,200,Food
29,416 Snack Bar,43.647511,-79.404018,187,Nightlife
19,Tequila Bookworm,43.647697,-79.401549,184,Nightlife
24,Isshin Ramen 一心,43.656517,-79.406945,179,Food
1,Cold Tea,43.654193,-79.401075,162,Nightlife
28,Bar Hop Brewco,43.649069,-79.393456,153,Nightlife


In [134]:
c.execute('DROP TABLE Foursquare_Overview')
POIs_df.to_sql('Foursquare_Overview', conn, if_exists='replace', index = False)
c.execute(''' SELECT * FROM Foursquare_Overview LIMIT 10''')
for row in c.fetchall():
     print (row)

("Sneaky Dee's Restaurant & Concert Venue", 43.65634906243686, -79.40764590401695, 407, 'Nightlife')
('Seven Lives - Tacos y Mariscos', 43.65441817160925, -79.40054467846261, 302, 'Food')
('Bar Raval', 43.65590380020168, -79.41012528837247, 266, 'Food')
('Horseshoe Tavern', 43.6493390929762, -79.3959763512841, 223, 'Nightlife')
("Jimmy's Coffee", 43.65449315540114, -79.40131090393002, 200, 'Food')
('416 Snack Bar', 43.64751101216232, -79.40401784574094, 187, 'Nightlife')
('Tequila Bookworm', 43.64769690944723, -79.40154896874483, 184, 'Nightlife')
('Isshin Ramen 一心', 43.65651655724417, -79.40694540739058, 179, 'Food')
('Cold Tea', 43.65419313416817, -79.40107491108753, 162, 'Nightlife')
('Bar Hop Brewco', 43.649068842850404, -79.39345567330301, 153, 'Nightlife')


In [ ]:
OIU====